In [ ]:
# this is a parameter that will get overwritten when run by papermill on a schedules
is_local_development = True

In [ ]:
!python -m pip install gitlabdata --upgrade

In [ ]:
!python -m pip install pyprojroot

In [ ]:
import configparser

# import sys
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
import json, os
from pyprojroot import here
from os import environ as env

In [ ]:
from gitlabdata.orchestration_utils import (
    data_science_engine_factory,
    query_dataframe,
    snowflake_engine_factory,
    snowflake_stage_load_copy_remove,
    get_env_from_profile,
    dataframe_uploader,
    write_to_gsheets,
    read_from_gsheets,
    query_executor,
    query_from_file,
)

## Create Snowflake engine

In [ ]:
# engine factory can be created using a local role from output
# depending on this notebook being run locally or remotely, the
# engine is creation process is different

if is_local_development:
    snowflake_engine = data_science_engine_factory(
        profile_target="sales_analytics_local"
    )
else:
    snowflake_engine = snowflake_engine_factory(env, "SALES_ANALYTICS")

    raw_db_name = env["SNOWFLAKE_LOAD_DATABASE"]
    prod_db_name = env["SNOWFLAKE_PROD_DATABASE"]

snowflake_engine

## Credentials for Gsheet manipulation

Remember to give access to the following two users:

- Data Team runner: data-team-sheets-sa@gitlab-analysis.iam.gserviceaccount.com
- Sales Strategy service account: service-revenue-strat-analytic@revenue-strategy-anal-411d5a72.iam.gserviceaccount.com

In [ ]:
# read the credentials of the google service account
if is_local_development:
    credentials_path = here("credentials/rsa_gcloud_service_account.json")
    # credentials_path = here("credentials/gsheet_service_file.json")

    with open(credentials_path) as f:
        service_account_credentials = f.read().replace("\n", "")

    # set the credential as a enviroment variable
    os.environ["GSHEETS_SERVICE_ACCOUNT_CREDENTIALS"] = service_account_credentials

# SAL Heatmap daily ETL update process

This notebook is run daily to update the SAL Heatmap source file. 

The process is responsible of updating the following datasets:

1. QTD SAL Extracts
2. Quotas
3. Regional Hist Coverages

In [ ]:
# SOURCE SAL Heatmap Sheet ID
SOURCE_GSHEET_ID = "1Ufh2DKwTq25IHzZeaCo0bFyQcFNvCtC3bVnR3oOZz-g"

## Excute Snowflake query

In [ ]:
# QTD metrics
qtd_metrics = query_from_file(snowflake_engine, "sh_qtd_metrics_for_sal_heatmap.sql")
qtd_metrics["last_extracted_etl"] = date.today()

In [ ]:
# Test
qtd_metrics.qtd_booked_net_arr.sum()

In [ ]:
# Historical coverages
hist_coverage = query_from_file(
    snowflake_engine, "sh_hist_qtr_coverages_for_sal_heatmap.sql"
)
hist_coverage["last_extracted_etl"] = date.today()

In [ ]:
# Quotas
# NF 20230531 The Xactly version of the quota capture doesn't work
# potentially data is not correct in Xactly
# fy24_quota = query_from_file(snowflake_engine, "sh_xactly_fy24_quotas.sql")

# NF 20230531 The table in this query is uploaded from a raw gsheet maintained by Haseem and
# uploaded into the raw sales analytics table
fy24_quota = query_from_file(snowflake_engine, "sh_raw_fy24_quotas.sql")
fy24_quota["last_extracted_etl"] = date.today()

In [ ]:
# Test
fy24_quota.cfy_total.sum()

In [ ]:
# Employee ID
empl_list = query_from_file(snowflake_engine, "sh_empid_list.sql")
empl_list["last_extracted_etl"] = date.today()

## QTD Metrics Extracts

### Write to SAL HEatmap source gSheet file

In [ ]:
# Write to GSheets
sheet_id = SOURCE_GSHEET_ID
sheet_name = "jupyter_qtd_sal"
qtd_metrics.columns = qtd_metrics.columns.str.upper()
write_to_gsheets(sheet_id, sheet_name, qtd_metrics)

## Historical QTR Coverages Data Extract

In [ ]:
# Write to GSheets
sheet_id = SOURCE_GSHEET_ID
sheet_name = "jupyter_hist_qtr_aggregated"
hist_coverage.columns = hist_coverage.columns.str.upper()
write_to_gsheets(sheet_id, sheet_name, hist_coverage)

## FY24 Quota

In [ ]:
# Write to GSheets
sheet_id = "1Pa3bh2wp1mPCtln0_3mH-r627IEJA5yofAFAir_kvBo"
sheet_name = "jupyter_fy_quota"
write_to_gsheets(sheet_id, sheet_name, fy24_quota)

## Employee List 

In [ ]:
# Write to GSheets
sheet_id = SOURCE_GSHEET_ID
sheet_name = "jupyter_empl_id"
write_to_gsheets(sheet_id, sheet_name, empl_list)